In [1]:
from pandas import read_csv, to_datetime,Timestamp,DateOffset, DataFrame, read_excel, ExcelWriter
import openpyxl

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
file = read_csv("./activity_summary_121224.csv")

In [3]:
lookup = read_excel("./lookup.xlsx",sheet_name="no_dup_temp")

In [4]:
# Convert 'Last Activity' to naive datetime (without timezone) for consistency
file['Last Activity'] = to_datetime(file['Last Activity'], errors='coerce').dt.tz_localize(None)


# Define the 12-month cutoff using naive datetime for consistency
#cutoff_date = Timestamp.now().tz_localize(None) - DateOffset(months=1)
oct_date_start = "2024-10-01"
oct_date_end = "2024-10-31"

nov_cutOff = "2024-11-01"

## Month activities

In [5]:
active_users_oct = file[(file['Last Activity'] >= oct_date_start) & (file['Last Activity'] <= oct_date_end)]
num_active_users_oct = len(active_users_oct)

active_users_nov = file[file['Last Activity'] >= nov_cutOff]
num_active_users_nov = len(active_users_nov)

In [6]:
num_active_users_oct
num_active_users_nov # keep

1

47

In [7]:
active = active_users_nov[['Username','Last Activity']]
analysis = active.merge(lookup,on="Username",how="left")

In [8]:
analysis.columns

Index(['Username', 'Last Activity', 'Full_Name', 'Key_stakeholders', 'groupe',
       'is_duplicated'],
      dtype='object')

In [9]:
usage_analysis = analysis.pivot_table(
    values= "is_duplicated",
    index = "Key_stakeholders",
    aggfunc= "count",
    fill_value=0,
    margins= True,
    margins_name="Total"
)
usage_analysis.rename(
    columns={
        "is_duplicated":"Number of Active Users since November 1 2024"
    },
    inplace=True
)


In [10]:
#usage_analysis
analysis.drop(columns=["is_duplicated"],axis=1, inplace=True)

In [11]:
#TODO add date du jour
mWater_usage_report = ExcelWriter(f"./mWater_usage_report_12/12/24.xlsx",engine="xlsxwriter")
usage_analysis.to_excel(mWater_usage_report,sheet_name="Summary",index=True,na_rep="")
analysis.to_excel(mWater_usage_report,sheet_name="List of Users",index=False,na_rep="")
mWater_usage_report.close()
